In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Replace with your actual chromedriver path
service = Service('/path/to/chromedriver')
driver = webdriver.Chrome(service=service)

url = "https://www.cars24.com/buy-used-renault-cars-mumbai/?sort=bestmatch&serveWarrantyCount=true&listingSource=Homepage_Filters"
driver.get(url)
time.sleep(5)  # wait for page JS to load

cars_data = []
car_cards = driver.find_elements(By.CSS_SELECTOR, 'div.cnPNdB')

for car in car_cards:
    try:
        model = car.find_element(By.TAG_NAME, 'h2').text.strip()
    except:
        model = 'N/A'

    try:
        price = car.find_element(By.CSS_SELECTOR, 'div.xexYZ').text.strip().replace('\n', ' ')
    except:
        price = 'N/A'

    details_elems = car.find_elements(By.CSS_SELECTOR, 'li.BjEfuq')
    year = 'N/A'
    km_driven = 'N/A'
    fuel = 'N/A'
    transmission = 'N/A'
    for d in details_elems:
        text = d.text.strip()
        if text.isdigit() and len(text) == 4:
            year = text
        elif 'km' in text.lower() or 'k' in text.lower():
            km_driven = text
        elif text in ['Petrol', 'Diesel', 'CNG', 'Electric']:
            fuel = text
        elif text in ['Manual', 'Automatic', 'Auto']:
            transmission = text

    try:
        location_full = car.find_element(By.CSS_SELECTOR, 'span._3hjfH').text.strip()
    except:
        location_full = 'Mumbai'

    rto_code = 'N/A'
    if ',' in location_full:
        parts = location_full.split(',')
        if parts[0].strip().startswith('MH'):
            rto_code = parts[0].strip()
            location_full = ",".join(parts[1:]).strip()
    else:
        for d in details_elems:
            if '-' in d.text and d.text[:2].isalpha():
                rto_code = d.text.strip()
                break

    cars_data.append({
        'Model': model,
        'Year of Manufacture': year,
        'KM Driven': km_driven,
        'Fuel Type': fuel,
        'Transmission': transmission,
        'Price': price,
        'RTO code': rto_code,
        'Location': location_full
    })

driver.quit()

df = pd.DataFrame(cars_data)
df.to_csv('use_it_recompiled', index=False)
print("Data scraped and saved to 'use_it_recompiled.csv'")

